In [146]:
# Add src directory to Python path
import sys
import os
from datetime import date, timedelta
import pandas as pd

sys.path.append(os.path.abspath('../src'))

from SnowflakeConnector import get_snowflake_session

from snowflake.snowpark.functions import date_trunc, current_date
from snowflake.snowpark.functions import col, lit, when
from snowflake.snowpark.types import DateType
import snowflake.snowpark.functions as f

# Get Snowflake session
session = get_snowflake_session()

In [147]:
df = session.table("analytics.analytics_inference.skio_subscriptions").where(col('frequency')=='bi-monthly')
df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CREATED_WEEK"  |"EMAIL"                     |"CUSTOMER_ID"  |"SUBSCRIPTIONID"                      |"FREQUENCY"  |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"CYCLESCOMPLETED"  |"STATUS"   |"STOREFRONTUSERID"                    |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-06-24      |louiselmbrown@gmail.com     |6935519461481  |c4272ee1-e7f2-48e8-8342-e53ff571b78d  |bi-monthly   |2024-06-24   |2024-09-14     |2024-09-09        |2                  |CANCELLED  |fe55b6fd-c7ce-4339-bf7f-f7717484384c  |
|2025-01-06      |louiselmbrown@gmail.com     |69355

In [122]:
today = date.today()
this_monday = today - timedelta(days=today.weekday())
mondays = [this_monday - timedelta(weeks=i+1) for i in range(9)]

snapshot_week = date(2025, 5, 19)  # or however you're setting it
week_plus_14 = snapshot_week + timedelta(days=14)


mondays

[datetime.date(2025, 6, 2),
 datetime.date(2025, 5, 26),
 datetime.date(2025, 5, 19),
 datetime.date(2025, 5, 12),
 datetime.date(2025, 5, 5),
 datetime.date(2025, 4, 28),
 datetime.date(2025, 4, 21),
 datetime.date(2025, 4, 14),
 datetime.date(2025, 4, 7)]

In [83]:
df.where(col("CREATED_WEEK")<snapshot_week)\
    .where((col("cancelled_week")>lit(snapshot_week))| (col("cancelledat").is_not_null())).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CREATED_WEEK"  |"EMAIL"                    |"CUSTOMER_ID"  |"SUBSCRIPTIONID"                      |"FREQUENCY"  |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"CYCLESCOMPLETED"  |"STATUS"   |"STOREFRONTUSERID"                    |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-09-09      |jillybean2213@hotmail.com  |7074824945769  |0d6ce8aa-eefa-4230-b222-bd10a361505e  |bi-monthly   |2024-09-14   |2024-12-10     |2024-12-09        |1                  |CANCELLED  |a72a4efa-7c7a-407c-861c-a4ebe7995b50  |
|2024-04-08      |carden0510@gmail.com       |6762652565

In [130]:


def build_snapshot (df, snapshot_week) :

    churn_labeled_snapshot = (
        df.where(col("CREATED_WEEK")<snapshot_week)\
        .where((col("cancelled_week")>lit(snapshot_week))| (col("cancelled_week").is_null()))\
            .select(
            lit(snapshot_week).alias("snapshot_week"),
            col("SUBSCRIPTIONID"),
            col("email"),
            col("createdat"),
            col("cancelledat"),
            col("cancelled_week"),
    
            f.dateadd('week', lit(2), col('snapshot_week')).alias("snapshot_week_plus_2"),
             f.datediff('week', col("CREATED_WEEK"), col("snapshot_week")).alias("lifetime"),
             f.datediff('day', col("createdat"), col("snapshot_week")).alias("lifetime_day"),

            when(
                #(col("cancelledat").is_not_null()) &
                #(col("cancelled_week") > col("snapshot_week")) &
                (col("cancelled_week") <= col("snapshot_week_plus_2")),
                1
            ).otherwise(0).alias("churn_label_14_day"),


            )
        )
    return churn_labeled_snapshot


In [131]:
all_snapshots = None

for week in mondays:
    snapshot = build_snapshot(df, week)
    all_snapshots = snapshot if all_snapshots is None else all_snapshots.union(snapshot)

    active = snapshot.agg(f.count_distinct(col('SUBSCRIPTIONID')).alias('active')).first()[0]
    churned = snapshot.agg(f.sum(col('churn_label_14_day')).alias('churned')).first()[0]
    print(f"Week Completed : {week}, Active: {active}, Churned: {churned}")

all_snapshots= all_snapshots.sort(['EMAIL', 'SUBSCRIPTIONID', 'SNAPSHOT_WEEK'])

Week Completed : 2025-06-02, Active: 11196, Churned: 0
Week Completed : 2025-05-26, Active: 11308, Churned: 340
Week Completed : 2025-05-19, Active: 11388, Churned: 698
Week Completed : 2025-05-12, Active: 11542, Churned: 741
Week Completed : 2025-05-05, Active: 11675, Churned: 715
Week Completed : 2025-04-28, Active: 11710, Churned: 617
Week Completed : 2025-04-21, Active: 12054, Churned: 888
Week Completed : 2025-04-14, Active: 12213, Churned: 1003
Week Completed : 2025-04-07, Active: 12328, Churned: 781


In [128]:
all_snapshots.show(20)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SNAPSHOT_WEEK"  |"SUBSCRIPTIONID"                      |"EMAIL"                                             |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"SNAPSHOT_WEEK_PLUS_2"  |"LIFETIME"  |"CHURN_LABEL_14_DAY"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-04-07       |1c8be995-05de-40ba-9525-b0bd92df2cf3  |00f1a9d5e0a8cb0c8e7e9bce003f2f751cdf4375fece7b2...  |2025-01-26   |NULL           |NULL              |2025-04-21              |11          |0                     |
|2025-04-14       |1c8be995-05de-40ba-9525-b0bd92df2cf3  |00f1a9d5e0a8cb0c8e7e9bce003f2f751cdf4375fece7b2...  |2

In [113]:
all_snapshots.where(col('EMAIL').isin(['lmgarrett8@gmail.com',
'lmgia25@aol.com',
'lmogol@gmail.com',
'deanna.dethloff@gmail.com',
'deanna.hizon@gmail.com',
'deb.crawford@gmail.com',
'deb6520@yahoo.com'])).show(50)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SNAPSHOT_WEEK"  |"SUBSCRIPTIONID"                      |"EMAIL"                    |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"SNAPSHOT_WEEK_PLUS_2"  |"LIFETIME"  |"CHURN_LABEL_14_DAY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-04-14       |dd239582-735d-47aa-93fc-7a370aca2736  |deanna.dethloff@gmail.com  |2024-10-07   |2025-05-05     |2025-05-05        |2025-04-28              |27          |0                     |
|2025-04-21       |dd239582-735d-47aa-93fc-7a370aca2736  |deanna.dethloff@gmail.com  |2024-10-07   |2025-05-05     |2025-05-05        |2025-05-05              |28          |1                     |
|2025-04-28    

In [118]:
all_snapshots.group_by('snapshot_week').agg(
    f.sum(col('churn_label_14_day')).alias('churned'),
    f.count(col('cancelledat')).alias('cancelled_count')    
    ).show()

---------------------------------------------------
|"SNAPSHOT_WEEK"  |"CHURNED"  |"CANCELLED_COUNT"  |
---------------------------------------------------
|2025-05-12       |741        |1072               |
|2025-04-21       |888        |2242               |
|2025-04-14       |1003       |2620               |
|2025-05-26       |340        |340                |
|2025-05-19       |698        |698                |
|2025-05-05       |715        |1394               |
|2025-06-02       |0          |0                  |
|2025-04-28       |617        |1664               |
---------------------------------------------------



In [141]:
# Get demographic features using Snowpark DataFrame API
demographic_features = session.table('analytics_inference.demographic_feature').select(
    'email',
    'males_per_100_females',
    'hh_mean_income',
    'hh_median_income', 
    'age_median',
    'AVG_HOUSEHOLD_SIZE',
    'MARRIED_HH',
    'SINGLE_MALE_HH',


)

# Display first few rows
demographic_features.show()


-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"EMAIL"                          |"MALES_PER_100_FEMALES"  |"HH_MEAN_INCOME"  |"HH_MEDIAN_INCOME"  |"AGE_MEDIAN"  |"AVG_HOUSEHOLD_SIZE"  |"MARRIED_HH"  |"SINGLE_MALE_HH"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|alisonway64@gmail.com            |102.0                    |93672             |59684               |36.9          |2.12                  |0.3212810     |0.2867133         |
|ktbelieveit@hotmail.com          |108.6                    |119149            |77209               |37.0          |2.82                  |0.2799084     |0.2052044         |
|jbwjenn@me.com                   |104.4                    |205792            |132494              |40.5          |2.20          

In [144]:
#join demographic features
enriched_snapshots = all_snapshots.join(
    demographic_features,
    all_snapshots['EMAIL'] == demographic_features['EMAIL'],
    'left'
).select(
    'SNAPSHOT_WEEK',
    'SUBSCRIPTIONID',
    all_snapshots['EMAIL'].alias('EMAIL'),  # Explicitly select EMAIL from all_snapshots
    'LIFETIME',
    'LIFETIME_DAY',
    'CHURN_LABEL_14_DAY',
    'MALES_PER_100_FEMALES',
    'HH_MEAN_INCOME',
    'HH_MEDIAN_INCOME',
    'AGE_MEDIAN',
    'AVG_HOUSEHOLD_SIZE',
    'MARRIED_HH',
    'SINGLE_MALE_HH'
)

enriched_snapshots = enriched_snapshots.sort(['EMAIL', 'SUBSCRIPTIONID', 'SNAPSHOT_WEEK'])
enriched_snapshots.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SNAPSHOT_WEEK"  |"SUBSCRIPTIONID"                      |"EMAIL"                                             |"LIFETIME"  |"LIFETIME_DAY"  |"CHURN_LABEL_14_DAY"  |"MALES_PER_100_FEMALES"  |"HH_MEAN_INCOME"  |"HH_MEDIAN_INCOME"  |"AGE_MEDIAN"  |"AVG_HOUSEHOLD_SIZE"  |"MARRIED_HH"  |"SINGLE_MALE_HH"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-04-07       |1c8be995-05de-40ba-9525-b0bd92df2cf3  |00f1a9d5e0a8cb0c8e7e9bce003f2f751

In [145]:
enriched_snapshots.\
        write.mode('overwrite') \
        .save_as_table('analytics.analytics_inference.bimonthly_churn_features')

In [59]:

churn_labeled_snapshot = build_snapshot (df, snapshot_week)

churn_labeled_snapshot.where(col('cancelledat').is_not_null()).order_by(col('cancelledat').asc()).show(20)


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SNAPSHOT_WEEK"  |"SUBSCRIPTIONID"                      |"EMAIL"                       |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"SNAPSHOT_WEEK_PLUS_2"  |"LIFETIME"  |"CHURN_LABEL_14_DAY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-05-19       |70ad08dd-67d4-427f-9493-2e44578ba6e1  |jkirallah07@gmail.com         |2025-05-05   |2025-05-26     |2025-05-26        |2025-06-02              |3           |1                     |
|2025-05-19       |c0e6effd-806c-4291-b8e0-4df29c19ca59  |tammyjohnston30068@yahoo.com  |2024-12-01   |2025-05-26     |2025-05-26        |2025-06-02              |26          |1                     |


In [136]:
# Join all_snapshots with demographic features and select specific columns
enriched_snapshots = all_snapshots.join(
    demographic_features,
    all_snapshots['EMAIL'] == demographic_features['EMAIL'],
    'left'
).select(
    'SNAPSHOT_WEEK',
    'SUBSCRIPTIONID',
    all_snapshots['EMAIL'].alias('EMAIL'),  # Explicitly select EMAIL from all_snapshots
    'CREATEDAT',
    'CANCELLEDAT',
    'CANCELLED_WEEK',
    'SNAPSHOT_WEEK_PLUS_2',
    'LIFETIME',
    'LIFETIME_DAY',
    'CHURN_LABEL_14_DAY',
    'MALES_PER_100_FEMALES',
    'HH_MEAN_INCOME',
    'HH_MEDIAN_INCOME',
    'AGE_MEDIAN'
)

# Show the first few rows of the joined data with single EMAIL column
print("Number of rows before join:", all_snapshots.count())
print("Number of rows after join:", enriched_snapshots.count())
enriched_snapshots.show()


Number of rows before join: 105414
Number of rows after join: 111438
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SNAPSHOT_WEEK"  |"SUBSCRIPTIONID"                      |"EMAIL"                  |"CREATEDAT"  |"CANCELLEDAT"  |"CANCELLED_WEEK"  |"SNAPSHOT_WEEK_PLUS_2"  |"LIFETIME"  |"LIFETIME_DAY"  |"CHURN_LABEL_14_DAY"  |"MALES_PER_100_FEMALES"  |"HH_MEAN_INCOME"  |"HH_MEDIAN_INCOME"  |"AGE_MEDIAN"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2025-05-12       |353a1cca-8fa0-45d7-b0b2-40cafae93